In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, cross_val_score, ParameterGrid
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
import numpy as np
np.int = int
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, RegressorMixin
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings("ignore")

#read input data
df_input = pd.read_csv('flow-cell.csv', index_col=0)

#filter input data
indices_to_remove_input = ['FE']
filtered_input = df_input.drop(indices_to_remove_input)
filtered_input = filtered_input.fillna(0) # fill  empty cells with 0
filtered_input = filtered_input.transpose() # change positions of columns and rows
filtered_input = filtered_input.loc[:, :'t [min]'] # remove columns after certain column
filtered_input = filtered_input.drop(['21065', '21090', '21091', '21097', '21092', '21080']) # remove outlier
filtered_input = filtered_input.drop(['21025', '21026', '21027', '21030', '21036']) # remove data without CE
#print(filtered_input)

#filter output data
df_output = pd.read_csv('flow-cell.csv', index_col=0)
filtered_output = df_output.loc['CE fuel [%]':, :,] # remove columns after certain column
filtered_output = filtered_output.fillna(0) # fill  empty cells with 0
filtered_output = filtered_output.transpose() # change positions of columns and rows
filtered_output = filtered_output.drop(['21065', '21090', '21091', '21097', '21092', '21080']) # remove outlier
filtered_output = filtered_output.drop(['21025', '21026', '21027', '21030', '21036']) # remove data without CE
#print(filtered_output)

X = filtered_input.values
Y = filtered_output[['CE fuel [%]', 'S decane w/o gas [%]','Ydecane total acid [%]']].values
#print(Y)

In [2]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=43)

# standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# normalize data with min-max scaling
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


knn_regressor = KNeighborsRegressor(
    n_neighbors=9,
    weights='distance',
    p=1
)


model=knn_regressor.fit(X_train, Y_train)


#_predict_test_knn = knn_regressor.predict(X_test)
#_predict_train_knn = knn_regressor.predict(X_train)

#se_training_knn = mean_squared_error(Y_train, Y_predict_train_knn)
#se_test_knn = mean_squared_error(Y_test, Y_predict_test_knn)

#rint(mse_training_knn)
#rint(mse_test_knn)

In [3]:
with open('knn_flow_CE-S-yield.sav', 'wb') as file:
    pickle.dump(model, file)